In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores.pgvector import PGVector, DistanceStrategy

from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

connection_string = PGVector.connection_string_from_db_params(
    driver=os.getenv("DB_DRIVER"),
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT'),
    database=os.getenv('DB_DATABASE'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD')
)

llm = OpenAI()
embeddings = OpenAIEmbeddings()

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader("../../state_of_the_union.txt")
docs = loader.load()

# loaders = [....]
# docs = []
# for loader in loaders:
#     docs.extend(loader.load())

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(docs)

In [2]:

collection_name = 'state_of_the_union'

# db = PGVector.from_documents(
#     embedding=embeddings,
#     documents=docs,
#     collection_name=collection_name,
#     connection_string=connection_string,
# )

db = PGVector(
    connection_string=connection_string, 
    embedding_function=embeddings, 
    collection_name=collection_name,
    distance_strategy=DistanceStrategy.COSINE
)

In [3]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [13]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), db.as_retriever())

In [14]:
query = "What did the president say about the economy?"
chat_history = []
result = qa({"question": query, "chat_history": chat_history, "vectordbkwargs": {"search_distance": 0.9}})

In [15]:
result["answer"]

' The president said that the economy has improved, with unemployment at a 50-year low, near record low unemployment for Black and Hispanic workers, and 800,000 good-paying manufacturing jobs created in the last two years. He also said that inflation is coming down, gas prices are down, and take home pay has gone up. He wants to continue to build an economy from the bottom up and the middle out.'